In [ ]:
from fastai.vision.all import *
import sys
import numpy as np


In [ ]:
ROOT = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
#IMAGE_OUTPUT = '../input/rsnamiccai-competition-2021-png-strips/'
IMAGE_OUTPUT = '.'

In [ ]:
import pandas as pd
import os
import random
df = pd.read_csv(os.path.join(ROOT, 'train_labels.csv'), header=0, names=['id','value'], dtype=object)
df = df[~df.id.isin(["00109", "00123", "00709"])]

In [ ]:
df.head()

In [ ]:
#https://stackoverflow.com/a/4836734/8245487
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [ ]:
np.maximum((1,5),(3,4))

In [ ]:
df_test = pd.DataFrame(columns=['id', 'value'])
df_test.id = os.listdir(os.path.join(ROOT, 'test'))

Use the cell below to process the DICOMs into PNG animation strips

In [ ]:
import os
import pydicom
import pandas as pd
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm
import binascii
from PIL import Image
from multiprocessing import Pool


if not os.path.exists('./train'):
    os.makedirs('./train')
    

if not os.path.exists('./test'):
    os.makedirs('./test')


def get_dicom_files(dataset='train'):
    root = f"{ROOT}/{dataset}"
    
    ids = list(df.id) if dataset=='train' else list(df_test.id)
    
    with Pool(10) as p:
        p.starmap(get_dicom_files_helper, [(dataset, root, cur_id) for cur_id in ids])
    

def get_dicom_files_helper(dataset, root, cur_id):
    final_image = np.array([])
    for scan_type in ['FLAIR', 'T1w', 'T1wCE', 'T2w']:
        cur_dir = os.path.join(root, cur_id, scan_type)
        dicoms = natural_sort(os.listdir(cur_dir))
        median = len(dicoms)//2
        modulo = max(len(dicoms)//20, 1)
        dicoms = [dicom for i, dicom in enumerate(dicoms) if i % modulo == 0]
        for dicom in dicoms:
            filepath = os.path.join(cur_dir, dicom)
            data = process_dicom(filepath)
            if len(final_image) != 0:
                (data_rows, data_cols) = data.shape
                (final_rows, final_cols) = final_image.shape
                if final_cols != data_cols:
                    if data_cols < final_cols:
                        data = np.hstack((data, np.zeros((data_rows, final_cols - data_cols))))
                    else:
                        final_image = np.hstack((final_image, np.zeros((final_rows, data_cols - final_cols))))
            final_image = np.concatenate([final_image,data]) if len(final_image) != 0 else data
            data = None

    outpath = os.path.join(f'./{dataset}',f'{cur_id}.png')
    write_pixels(final_image, outpath)
    final_image = None
    print('Processed {}'.format(cur_id))
        

def process_dicom(path):
    dicom = pydicom.read_file(path)
    data = apply_voi_lut(dicom.pixel_array, dicom)
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

    
def write_pixels(data, outpath):
    height = len(data)
    width = len(data[0])
    pixels_out = []
    for row in data:
        pixels_out.extend(row)
    assert(len(pixels_out) == height * width)
    
    image_out = Image.new('L', (width, height))
    image_out.putdata(pixels_out)
    image_out.save(outpath)


In [ ]:
get_dicom_files('train')
get_dicom_files('test')

In [ ]:
for id_num in df.id:
    full_path = os.path.join(IMAGE_OUTPUT, 'train/{}.png'.format(id_num))
    df.loc[df.id == id_num, 'file'] = full_path
    

In [ ]:
df

In [ ]:
dls = ImageDataLoaders.from_df(df, item_tfms=Resize((40000,1024),method='pad',pad_mode=PadMode.Zeros), bs=2, label_col =1, fn_col=2, valid_pct=0.05)

In [ ]:
dls.show_batch()

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        #self.conv2 = nn.Conv2d(6, 6, 5)
        self.innerConvLayers = nn.ModuleList()
        for i in range(15):
            self.innerConvLayers.append(nn.Conv2d(6, 6, 5))
        self.fc1 = nn.Linear(6 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.leaky_relu_(self.conv1(x)))
        for layer in self.innerConvLayers:
            x = self.dropout(self.pool(F.leaky_relu_(layer(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.leaky_relu_(self.fc1(x))
        x = F.leaky_relu_(self.fc2(x))
        x = F.leaky_relu_(self.fc3(x))
        return x

In [ ]:
print(Net())

In [ ]:


learn = cnn_learner(dls, Net, metrics=[error_rate, accuracy], model_dir="/tmp/model/").to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, lr_max=1e-2)

In [ ]:
learn.show_results()

In [ ]:
for id_num in df_test.id:
    full_path = os.path.join(IMAGE_OUTPUT, 'test/{}.png'.format(id_num))
    prediction = learn.predict(full_path)
    print(prediction)
    probability = prediction[2][1].item()
    print(probability)
    df_test.loc[df_test.id==id_num, 'value'] = probability

In [ ]:
df_test.head()

In [ ]:
df_test.value.min()

In [ ]:
df_test.value.max()

In [ ]:
df_test.rename(columns={'id':'BraTS21ID','value':'MGMT_value'}).to_csv('submission.csv', index=False)